 # Połączenie z bazą danych przy pomocy psycopg2 i pandas

In [1]:
import psycopg2 as pg
import pandas as pd


host = "pgsql-196447.vipserv.org"
port = 5432
dbname = "wbauer_adb"
user = "wbauer_adb"
password = "adb2020"

connection = pg.connect(host=host, port=port, dbname=dbname, user=user, password=password)

***Wykonaj zapytania za pomocą skryptów Pythona.***

1. Ile kategorii filmów mamy w wypożyczalni?

In [171]:
pd.read_sql("SELECT COUNT(DISTINCT name) AS categories FROM category;", con=connection)

,categories
0,16


W wypożyczalni mamy 16 kategorii filmów.

2. Wyświetl listę kategorii w kolejności alfabetycznej.

In [57]:
pd.read_sql("SELECT DISTINCT name FROM category ORDER BY name ASC;", con=connection)

,name
0,Action
1,Animation
2,Children
3,Classics
4,Comedy
5,Documentary
6,Drama
7,Family
8,Foreign
9,Games


3. Znajdź najstarszy i najmłodszy film do wypożyczenia.

In [58]:
pd.read_sql("SELECT title, release_year FROM film ORDER BY release_year ASC LIMIT 1;", con=connection)

,title,release_year
0,Chamber Italian,2006


In [77]:
film_table = pd.read_sql(f"SELECT * FROM film;", con=connection)
release_year_cond = f"SELECT title, release_year FROM film ORDER BY release_year ASC LIMIT 1 OFFSET {len(film_table) - 1};"
pd.read_sql(release_year_cond, con=connection)

,title,release_year
0,Zorro Ark,2006


4. Ile wypożyczeń odbyło się między 2005-07-01 a 2005-08-01?

In [170]:
date_sql = "SELECT COUNT(*) AS rentals FROM rental WHERE rental_date BETWEEN '2005-07-01 00:00:00' AND '2005-08-01 23:59:59';"
pd.read_sql(date_sql, con=connection)

,rentals
0,7380


Między 2005-07-01 a 2005-08-01 miało miejsce 7380 wypożyczeń.

5. Ile wypożyczeń odbyło się między 2010-01-01 a 2011-02-01?

In [169]:
date_sql = "SELECT COUNT(*) AS rentals FROM rental WHERE rental_date BETWEEN '2010-01-01 00:00:00' AND '2011-02-01 23:59:59';"
pd.read_sql(date_sql, con=connection)

,rentals
0,0


Między 2010-01-01 a 2011-02-01 nie miało miejsca żadne wypożyczenie.

6. Znajdź największą płatność wypożyczenia.

In [31]:
pd.read_sql("SELECT amount FROM payment ORDER BY amount DESC LIMIT 1;", con=connection)

,amount
0,11.99


7. Znajdź wszystkich klientów z Polski, Nigerii lub Bangladeszu.

In [141]:
sql = "SELECT DISTINCT first_name, last_name, email\
       FROM customer\
       WHERE address_id\
       IN (SELECT address_id\
           FROM address\
           WHERE city_id\
           IN (SELECT city_id\
               FROM city\
               WHERE country_id\
               IN (SELECT country_id\
                   FROM country\
                   WHERE country='Poland'\
                   OR country='Nigeria'\
                   OR country='Bangladesh')));"

pd.read_sql(sql, con=connection)

,first_name,last_name,email
0,Sidney,Burleson,sidney.burleson@sakilacustomer.org
1,Stephen,Qualls,stephen.qualls@sakilacustomer.org
2,Elsie,Kelley,elsie.kelley@sakilacustomer.org
3,Carol,Garcia,carol.garcia@sakilacustomer.org
4,Frank,Waggoner,frank.waggoner@sakilacustomer.org
5,Russell,Brinson,russell.brinson@sakilacustomer.org
6,Brian,Wyman,brian.wyman@sakilacustomer.org
7,Marilyn,Ross,marilyn.ross@sakilacustomer.org
8,Sonia,Gregory,sonia.gregory@sakilacustomer.org
9,Gladys,Hamilton,gladys.hamilton@sakilacustomer.org


8. Gdzie mieszkają członkowie personelu?

In [168]:
sql = "SELECT DISTINCT country\
       FROM country\
       WHERE country_id\
       IN (SELECT country_id\
           FROM city\
           WHERE city_id\
           IN (SELECT city_id, \
               FROM address\
               WHERE address_id\
               IN (SELECT address_id\
                   FROM staff)));"


pd.read_sql(sql, con=connection)

,country
0,Canada
1,Australia


Członkowie personelu mieszkają w Australii i Kanadzie.

9. Ilu pracowników mieszka w Argentynie lub Hiszpanii?

In [172]:
sql = "SELECT COUNT(*) AS argentinian_or_spanish_employees\
       FROM (SELECT DISTINCT first_name, last_name\
             FROM staff\
             WHERE address_id\
             IN (SELECT address_id\
                 FROM address\
                 WHERE city_id\
                 IN (SELECT city_id\
                     FROM city\
                     WHERE country_id\
                     IN (SELECT country_id\
                         FROM country\
                         WHERE country='Argentina'\
                         OR country='Spain')))) AS _;"

pd.read_sql(sql, con=connection)

,argentinian_or_spanish_employees
0,0


Żaden pracownik nie mieszka w Argentynie lub Hiszpanii.

10. Jakie kategorie filmów zostały wypożyczone przez klientów?

In [177]:
sql = "SELECT DISTINCT name\
       FROM category\
       WHERE category_id\
       IN (SELECT category_id\
           FROM film_category\
           WHERE film_id\
           IN (SELECT film_id\
               FROM film\
               WHERE film_id\
               IN (SELECT film_id\
                   FROM inventory\
                   WHERE inventory_id\
                   IN (SELECT inventory_id\
                       FROM rental))));"

pd.read_sql(sql, con=connection)

,name
0,Action
1,Animation
2,Children
3,Classics
4,Comedy
5,Documentary
6,Drama
7,Family
8,Foreign
9,Games


11. Znajdź wszystkie kategorie filmów wypożyczonych w Ameryce.

In [180]:
# sql = "SELECT DISTINCT name\
#        FROM category\
#        WHERE category_id\
#        IN (SELECT category_id\
#            FROM film_category\
#            WHERE film_id\
#            IN (SELECT film_id\
#                FROM film\
#                WHERE film_id\
#                IN (SELECT film_id\
#                    FROM inventory\
#                    WHERE inventory_id\
#                    IN (SELECT inventory_id\
#                        FROM rental\
#                        WHERE staff_id\
#                        IN (SELECT staff_id\
#                            FROM staff\
#                            WHERE store_id\
#                            IN (SELECT store_id\
#                                FROM store\
#                                WHERE address_id\
#                                IN (SELECT address_id\
#                                    FROM address\
#                                    WHERE city_id\
#                                    IN (SELECT city_id\
#                                        FROM city\
#                                        WHERE country_id\
#                                        IN (SELECT country_id\
#                                            FROM country\
#                                            WHERE country='United States')))))))))"

sql = "SELECT DISTINCT name\
       FROM category\
       WHERE category_id\
       IN (SELECT category_id\
           FROM film_category\
           WHERE film_id\
           IN (SELECT film_id\
               FROM film\
               WHERE film_id\
               IN (SELECT film_id\
                   FROM inventory\
                   WHERE inventory_id\
                   IN (SELECT inventory_id\
                       FROM rental\
                       WHERE staff_id\
                       IN (SELECT staff_id\
                           FROM staff\
                           WHERE store_id\
                           IN (SELECT store_id\
                               FROM store\
                               WHERE address_id\
                               IN (SELECT address_id\
                                   FROM address\
                                   WHERE city_id\
                                   IN (SELECT city_id\
                                       FROM city\
                                       WHERE country_id\
                                       IN (SELECT country_id\
                                           FROM country\
                                           WHERE country='United States')))))))))"

pd.read_sql(sql, con=connection)

,name


12. Znajdź wszystkie tytuły filmów, w których grał: Olympia Pfeiffer lub Julia Zellweger lub Ellen Presley.

In [183]:
sql = "SELECT title\
       FROM film\
       WHERE film_id\
       IN (SELECT film_id\
           FROM film_actor\
           WHERE actor_id\
           IN (SELECT actor_id\
               FROM actor\
               WHERE (first_name='Olympia'\
               AND last_name='Pfeiffer')\
               OR (first_name='Julia'\
               AND last_name='Zellweger')\
               OR (first_name='Ellen'\
               AND last_name='Presley')));"

pd.read_sql(sql, con=connection)

,title
0,Caribbean Liberty
1,Roots Remember
2,Outlaw Hanky
3,Women Dorado
4,Empire Malkovich
...,...
64,Contact Anonymous
65,Majestic Floats
66,Deep Crusade
67,Effect Gladiator
